# Исследование надёжности заёмщиков

#### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### План действий

1. [Изучить файл](#step1)
2. [Предобработать данные](#step2)
   * [Обработать пропуски](#step2.1)  
   * [Заменить типы данных](#step2.2)
   * [Обработать дубликаты](#step2.3)
   * [Лемматизация данных](#step2.4)
   * [Категоризация данных](#step2.5)
3. [Ответы на вопросы](#step3)
   * [Есть ли зависимость между возвратом кредита в срок и наличием детей](#step3.1)
   * [Есть ли зависимость между возвратом кредита в срок и семейным положением](#step3.2)
   * [Есть ли зависимость между возвратом кредита в срок и уровнем дохода](#step3.3)
   * [Есть ли зависимость между возвратом кредита в срок и целью кредита](#step3.4)
4. [Общие выводы](#step4)

<a name="step1"></a>
### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [3]:
clients = pd.read_csv('/datasets/data.csv')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

#### Типы данных
В полученном нами датасете есть некоторые несостыковки с физическим смыслом и типом данных:
* days_employed (трудовой стаж в днях) - данные, которые должны быть неотрицательными и целочисленным, то есть int64. На деле мы видим, что поле имеет тип float64, что может подразумевать как дробные значения, там и отрицательные числа.
* debt (была ли задолженность по возврату кредитов) - мог быть bool, но в целом и тип int64 подходит для этих целей (1 = да, 0 = нет)

Данный факт указывает на то, что были ошибки в сборе данных или же в их выгрузке

#### Пропуски в данных
В данных содержатся пропуски, в полях days_employed (трудовой стаж в днях) и в total_income (ежемесячный доход). Если человек ни разу не работал (ребенок, студент), то у него соответственно будет отсутствовать трудовой стаж и доход. Но для студентов и пенсионеров предусмотрена стипендия (при этом студент дожен хорошо учиться), поэтому ежемесячный доход у таких групп людей должен присутсвовать.

Но если у человека имеется занятость в виде работы (аналитик, разработчик и тд), то у таких людей должен быть заработок, а также трудовой стаж. Иначе было место ошибке в сборе/выгрузке данных.

#### Противоречие в данных
- Отрицательный трудовой стаж
- Дробный трудовой стаж
- Слишком большой трудовой стаж (340 тыс дней)

#### Категории данных
+ Встречаются различные написания одного и того же образования (education)
+ Встречаются различные формулировки цели кредита (purpose)


<a name="step2"></a>
### Шаг 2. Предобработка данных

<a name="step2.1"></a>
### Обработка пропусков

Проверим, у всех ли записей без трудового стажа отсутствует ежемесячный доход

In [5]:
filter_never_working = clients['days_employed'].isnull()
filter_not_earn = clients['total_income'].isnull()
len(clients[filter_not_earn & filter_never_working])

2174

Когда выполняются оба условия (нет стажа и дохода), количество строк  2174 соответствует количеству строк с отсутствующими данными каждого условия по-отдельности (21525 - 19351). 
Поэтому, скорее всего, пользователям не высветились оба поля ввода

<a name="empty"></a>
Посмотрим на первые 10 строк данных, в котрых стоят пропуски по трудовому стажу и ежемесячному доходу

In [6]:
never_working = clients[clients['days_employed'].isnull()]
never_working.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Среди данных мы видим, что у человека имеется работа, но при этом его трудовой стаж в днях отсутсвует и ежемесячный доход тоже.
Рассмотрим, какие бывают у таких людей типы занятости

In [7]:
never_working['income_type'].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

- Пенсионер мог никогда не работать (видимо получил наследство от своей бабушки), тогда и пенсию ему не с чего получать
- Госслужащий, сотрудник абсолютно точно должны иметь трудовой стаж и ежемесячное жалованье
- Предприниматель, возможно, считает, что на его любимом деле он никогда не работал, но заработок быть должен
- Компаньон, похож на предпринимателя (только делит компанию с кем-то еще)

Проверим, есть ли трудовой стаж у пенсионера, предпринимателя, компаньона

In [ ]:
ever_working = clients[clients['days_employed'].notnull()]
ever_working['income_type'].unique()

<a name="fill_empty"></a>
Мы видим, что в нормальной жизни (то есть, в 19351 случае собранных данных) пенсионер, предприниматель, компаньон имеют трудовой стаж и ещемесячный доход. Также как его имеют и остальные типы занятых людей без данных о стаже и доходе.

Чтобы аккуратно заполнить пропуски, будем брать среднее по роду деятельности в настоящее время. Но, на доход в прямой пропорциональности влияет трудовой стаж, а на трудовой стаж влияет возраст человека (чем больше человеку лет, тем больший трудовой стаж он имеет). 

Также, в мире есть преобладание женских профессий и мужских (у них и доходы, кстати, отличаются). Поэтому пол также будет фактором влияния.

Ко всему этому следует добавить, что для женщин на трудовой стаж оказывает дополнительное влияние количество детей (приходится уходить в декрет, повышение по службе откладывается, доходы тоже не быстро растут).

Поэтому будет считать среднее по возрасту, роду деятельности, полу и количеству детей (только для женского пола).

In [9]:
# Напишем функцию подсчитывающую среднее значение по всем колонкам со влияющими параметрами: 
# возраст, роде деятельност, пол и количество детей (последний параметр влияет только на женщин)
# В случае, если с такими параметрами будет пустой датафрейм, 
# то функция будет возвращать среднее значение по всему датафрейму

def mean_with_factors(row):
    '''
    Функция возвращающая срдение численные значения датафрейма
    с текущим возрастом, родом деятельности, полом и количеством детей (только для женщин)
    '''
    # текущие значения параметров данного человека
    dob_years = row['dob_years']
    income_type = row['income_type']
    children = row['children']
    gender = row['gender']
    
    # фильтры для отбора строк
    filter_years = clients['dob_years'] == dob_years
    filter_type = clients['income_type'] == income_type
    filter_gender = clients['gender'] == gender
    filter_children = True
    if gender == 'F':
        filter_children = clients['children'] == children
    
    # среднее значение трудового стажа
    mean = clients[filter_years & filter_type & filter_gender & filter_children].mean()
    if mean['total_income'] != mean['total_income']: # проверка на NaN
        return clients.mean()
    return mean
    

In [11]:
# применим вышеописанную функцию, чтобы заполнить пропуски в трудовом стаже и ежемесячном доходе
clients.loc[clients['days_employed'].isnull(), 'days_employed'] = clients[clients['days_employed'].isnull()].apply(mean_with_factors, axis=1)['days_employed'].astype('int')
clients.loc[clients['total_income'].isnull(), 'total_income'] = clients[clients['total_income'].isnull()].apply(mean_with_factors, axis=1)['total_income'].astype('int')

In [12]:
# Проверим, заполнились ли пропуски
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

<b>Где пропуски</b>:
У всех записей с отсутствием дней трудового стажа (days_employed) также отсутствует и ежемесячный доход (total_income) и наоборот.

<b>Причины</b>: Не до конца заполнились бланки, оборудование плохо считало.

<b>Как заполнила</b>:
    Такие пропуски я заполнила средним значением из выборки людей с таким же возрастом, родом деятельности, пола,   количесвтом детей (для женщин). В случае отсутсвия подобной выборки, среднее бралось из всего объема данных.

<a name="step2.2"></a>
### Замена типа данных

Рассмотрим еще раз первые несколько строк датафрейма.

Замена типа данных требуется:
- days_employed (трудовой стаж в днях) встречаются отрицательные дни и дробные
- total_income (ежемесячный доход) заменим на int (копейки не сильно нам важны, а на целочисленный тип требуется меньше памяти)
- также стоит обратить внимание на children	(количество детей), там могут быть отрицательные значения и слишком большие
- dob_years (возраст) аналогично предыдущему высказыванию
- debt (была ли задолженность) должно быть два возможных значения 1 или 0

In [13]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<a name="children"></a>
#### Замена данных с детьми

In [14]:
# различные значения количества детей
clients['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

<a name="change_children"></a>
Для детей точно требуется замена данных.   
Скорее всего ввод производится с помощью распознавания ручных записей, поэтому легкий штрих сделал у человека -1 ребенка (подразумевался 1), а точка в конце могла превратить 2 ребенка в 20 детей.  
Исправим этот момент

In [16]:
# замена
clients.loc[clients['children'] == 20, 'children'] = 2
clients.loc[clients['children'] == -1, 'children'] = 1

# проверка выполнения
clients['children'].unique()

array([1, 0, 3, 2, 4, 5])

<a name="change_income"></a>
#### Замена ежемесячного дохода на int

In [17]:
# замена типа данных
clients['total_income'] = clients['total_income'].astype(int)

# проверка выполнения
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


#### Замена данных с возрастом

In [18]:
clients['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

<a name="age"></a>
Есть данные, где у человека возраст равен 0, но кредиты нельзя брать людям младше 18 лет.

In [19]:
# Посмотрим, сколько таких записей
clients[clients['dob_years'] < 18]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,-3061.000000,0,среднее,1,женат / замужем,0,F,сотрудник,0,125680,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


<a name="delete_age"></a>
Всего 101 запись. Определять их возраст на основе других записей, может выйти дольше, поэтому просто избавимся от этих значений.

In [20]:
# избавляемся от возраста < 18
clients = clients[clients['dob_years'] >= 18].reset_index(drop=True)

# проверим, сколько записей отсалось
len(clients)

21424

#### Замена данных с трудовым стажем

<a name="days_employed"></a>
Посмотрим статистическую информацию по трудовому стажу

In [21]:
clients['days_employed'].describe()

count     21424.000000
mean      63336.841920
std      140982.512667
min      -18388.949901
25%       -2754.000000
50%       -1306.916190
75%        -314.358442
max      401755.400475
Name: days_employed, dtype: float64

Мы видим, что основную часть составляют отрицательные значения (более 75%). И данные отрицательные значения соответсвуют реальному трудовому стажу в днях (самое большое 18 тыс дней = 65 лет), но для положительных чисел наблюдаются слишком завышенные значения (400 тыс дней = 1481 год). В среднем, слишком большой трудовой стаж больше реального трудового стажа примерно в 24 раза. Поэтому, можно сделать вывод о том, что некоторые написали свой трудовой стаж в часах.

<a name="change_days_employed"></a>
Поэтому, числа большие 0 необходимо разделить на 24, из отрицательных взять модуль и весь столбец сделать типа int

In [31]:
clients.loc[clients['days_employed'] > 0, 'days_employed'] = clients['days_employed'] / 24
# избавимся от отрицательных и дробных значений (заменив тип переменной на int и взяв модуль числа)

clients['days_employed'] = clients['days_employed'].abs().astype(int)

# проверка выполнения
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

In [24]:
clients.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Были выявлены следующие проблемы:

1) Количество детей в данных было равно -1 и 20. Это могло возникнуть в  связи с плохим распознаванием ручных записей (-1 ребенок = 1 ребенок, 20 детей = 2 детей)

2) Ежемесяный доход был с высокой точностью, что не является необходимым (округлили доход до целого применив метод .astype(int)

3) Возраст людей был равен 0 (таких данных было всего 101 запись, поэтому просто от них избавились)

4) Трудовой стаж в днях содержал дробные числа и отрицатедьные значения (пришлось взять модуль числа и отбросить дробную часть методом .astype(int)

<a name="step2.3"></a>
### Обработка дубликатов

In [33]:
len(clients)

21424

In [34]:
clients = clients.drop_duplicates().reset_index(drop=True)
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21370 entries, 0 to 21369
Data columns (total 12 columns):
children            21370 non-null int64
days_employed       21370 non-null int64
dob_years           21370 non-null int64
education           21370 non-null object
education_id        21370 non-null int64
family_status       21370 non-null object
family_status_id    21370 non-null int64
gender              21370 non-null object
income_type         21370 non-null object
debt                21370 non-null int64
total_income        21370 non-null int64
purpose             21370 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


В данных содержались дубликаты строк. Размер датафрейма уменьшился с 21424 до 21370 строк.

В некоторых полях принимаемые значения в строке тоже дублируются.  
Посмотрим на дубли значений поле семейный статус

In [35]:
clients['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Все статусы уникальные, с ними ничего делать не надо

<a name="education"></a>
Рассмотрим теперь, уникальные значения образования

In [36]:
clients['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Среди них встречаются одинаковые значения ('неоконченное высшее' и 'НЕОКОНЧЕННОЕ ВЫСШЕЕ').

<a name="change_education"></a>
Здесь можно привести все к нижнему регистру

In [37]:
# изменение данных
clients['education'] = clients['education'].str.lower()

# проверим изменения и с учетом education_id
clients[['education', 'education_id']].drop_duplicates()

,education,education_id
0,высшее,0
1,среднее,1
13,неоконченное высшее,2
31,начальное,3
2946,ученая степень,4


### Вывод

#####  Дубликаты: 

Встречаются одинаковые значения в поле education (образование)

##### Причины:

С разных платформ отправляется либо в верхнем регистре, нижнем, capitalize. Видно, что не ручная ошибка (скорее всего такой шаблон в приложении)

##### Как избавилась:

Привела все записи к нижнему регистру.

<a name="step2.4"></a>
### Лемматизация

В столбце с целью кредита содержатся нелемматизированные значения, которые дают большое количество уникальных

In [38]:
clients['purpose'].nunique()

38

In [39]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# лемматизируем весь столбец, сделав из него одну строку
text = clients['purpose'].str.cat(sep=' ')
lemmas = m.lemmatize(text)

# подсчитаем количество повторябщихся лемм
count = Counter(lemmas)

In [40]:
count

Counter({'покупка': 5873,
         ' ': 54830,
         'жилье': 4437,
         'приобретение': 459,
         'автомобиль': 4286,
         'дополнительный': 903,
         'образование': 3996,
         'сыграть': 764,
         'свадьба': 2321,
         'операция': 2593,
         'с': 2904,
         'на': 2216,
         'проведение': 769,
         'для': 1287,
         'семья': 637,
         'недвижимость': 6330,
         'коммерческий': 1307,
         'жилой': 1225,
         'строительство': 1874,
         'собственный': 633,
         'подержать': 849,
         'свой': 2224,
         'со': 627,
         'заниматься': 900,
         'сделка': 938,
         'получение': 1310,
         'высокий': 1366,
         'подержанный': 113,
         'профильный': 435,
         'сдача': 650,
         'ремонт': 605,
         '\n': 1})

### Вывод

Лемматирировав поле цель кредита, получилось выделить основные цели кредита:

1) недвижимость

2) автомобиль

3) образование

4) свадьба

5) ремонт

<a name="step2.5"></a>
### Категоризация данных

Для ответов на вопросы ниже, необходимо произвести категоризацию данных по столбцам: цель кредита, уровень дохода.

<a name="cat_purpose"></a>
Категории для цели кредита были определены на предыдущем шаге ([лемматизация](#step2.4)). Выделим эти категории в отдельное поле

In [41]:
# Выделим категорию в столбце цель кредита: 
# недвижимость, автомобиль, образование, свадьба, ремонт

def category_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    if 'ремонт' in lemmas:
        return 'ремонт'
    if 'недвижимость' in lemmas or 'жилье' in lemmas or 'строительство' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    return 'другое'

In [42]:
clients['category_purpose'] = clients['purpose'].apply(category_purpose)
clients['category_purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба', 'ремонт'],
      dtype=object)

<a name="cat_income"></a>
Категоризируем доход исходя из квантилей (25%, 50%, 75%)

Найдем эти значения в каждом из квантилей

In [61]:
pd.set_option('display.float_format', lambda x: '%.3f' % x) # уберем отображение e+

In [62]:
clients['total_income'].describe()

count     21370.000
mean     167402.229
std       98465.355
min       20667.000
25%      107411.500
50%      148243.000
75%      200114.750
max     2265604.000
Name: total_income, dtype: float64

In [63]:
# напишем функцию которая будет возвращать категорию исходя из того, в каком квантиле лежит доход человека

def category_income(income):
    '''
    Функция, определяющая категорию по ежемесячному доходу
    '''
    if income < 107411.5:
        return '0-25%'
    if income < 148243.0:
        return '26-50%'
    if income < 200114.75:
        return '51-75%'
    return '76-100%'

In [64]:
clients['category_income'] = clients['total_income'].apply(category_income)
clients['category_income'].unique()

array(['76-100%', '26-50%', '51-75%', '0-25%'], dtype=object)

### Вывод

Проведена категоризация по одному столбцу (по количеству детей, доходу, цели кредита). 
Для целей кредита выделен словарь с самыми распространенными целями кредита. Для остальных столбцов применила количественное сравнение и определение группы (высокий дохо, нет детей)

<a name="step3"></a>
### Шаг 3. Ответьте на вопросы

<a name="step3.1"></a>
- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [68]:
children_debt = clients.pivot_table(values='gender', columns='debt', index='children', aggfunc='count')

children_debt.columns = ['В срок', 'Просрочил']
children_debt['Всего'] = children_debt['В срок'] + children_debt['Просрочил']
children_debt['% просроченных'] = children_debt['Просрочил'] / children_debt['Всего']

children_debt

,В срок,Просрочил,Всего,% просроченных
children,,,,
0,12980.000,1058.000,14038.000,0.075
1,4398.000,442.000,4840.000,0.091
2,1912.000,202.000,2114.000,0.096
3,301.000,27.000,328.000,0.082
4,37.000,4.000,41.000,0.098
5,9.000,nan,nan,nan


### Вывод

При отсутствии детей наблюдается самое низкое количество просроченных долгов: 7,5%
Для 1,2,4 детей - одни из самых высоких показателей: более 9%

(Видимо, 1 и 2 детей имеют молодые семьи, которые не всегда справляются со всем; а 4 ребенка - это уже совсем перебор.)

<a name="step3.2"></a>
- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [100]:
data.groupby(['family_status', 'debt']).count()

children  days_employed  dob_years  education  \
family_status         debt                                                  
Не женат / не замужем 0         2524           2524       2524       2524   
                      1          273            273        273        273   
в разводе             0         1100           1100       1100       1100   
                      1           85             85         85         85   
вдовец / вдова        0          893            893        893        893   
                      1           62             62         62         62   
гражданский брак      0         3770           3770       3770       3770   
                      1          386            386        386        386   
женат / замужем       0        11404          11404      11404      11404   
                      1          927            927        927        927   

                            education_id  family_status_id  gender  \
family_status         debt                                           
Не женат / не замужем 0             2524              2524    2524   
                      1              273               273     273   
в разводе             0             1100              1100    1100   
                      1               85                85      85   
вдовец / вдова        0              893               893     893   
                      1               62                62      62   
гражданский брак      0             3770              3770    3770   
                      1              386               386     386   
женат / замужем       0            11404             11404   11404   
                      1              927               927     927   

                            income_type  total_income  purpose  \
family_status         debt                                       
Не женат / не замужем 0            2524          2524     2524   
                      1             273           273      273   
в разводе             0            1100          1100     1100   
                      1              85            85       85   
вдовец / вдова        0             893           893      893   
                      1              62            62       62   
гражданский брак      0            3770          3770     3770   
                      1             386           386      386   
женат / замужем       0           11404         11404    11404   
                      1             927           927      927   

                            category_purpose  category_children  \
family_status         debt                                        
Не женат / не замужем 0                 2524               2524   
                      1                  273                273   
в разводе             0                 1100               1100   
                      1                   85                 85   
вдовец / вдова        0                  893                893   
                      1                   62                 62   
гражданский брак      0                 3770               3770   
                      1                  386                386   
женат / замужем       0                11404              11404   
                      1                  927                927   

                            category_income  
family_status         debt                   
Не женат / не замужем 0                2524  
                      1                 273  
в разводе             0                1100  
                      1                  85  
вдовец / вдова        0                 893  
                      1                  62  
гражданский брак      0                3770  
                      1                 386  
женат / замужем       0               11404  
                      1                 927

In [67]:
family_debt = clients.pivot_table(values='gender', columns='debt', index='family_status', aggfunc='count')

family_debt.columns = ['В срок', 'Просрочил']
family_debt['Всего'] = family_debt['В срок'] + family_debt['Просрочил']
family_debt['% просроченных'] = family_debt['Просрочил'] / family_debt['Всего']

family_debt

,В срок,Просрочил,Всего,% просроченных
family_status,,,,
Не женат / не замужем,2521,273,2794,0.098
в разводе,1100,85,1185,0.072
вдовец / вдова,892,62,954,0.065
гражданский брак,3756,386,4142,0.093
женат / замужем,11368,927,12295,0.075


### Вывод

Зависимости между наличием детей и задолженности нет.

В всех случаях наблюдается низкая задолженность (от 6 до 10%)

<a name="step3.3"></a>
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [69]:
income_debt = clients.pivot_table(values='gender', columns='debt', index='category_income', aggfunc='count')

income_debt.columns = ['В срок', 'Просрочил']
income_debt['Всего'] = income_debt['В срок'] + income_debt['Просрочил']
income_debt['% просроченных'] = income_debt['Просрочил'] / income_debt['Всего']

income_debt

,В срок,Просрочил,Всего,% просроченных
category_income,,,,
0-25%,4917,426,5343,0.080
26-50%,4876,460,5336,0.086
51-75%,4879,469,5348,0.088
76-100%,4965,378,5343,0.071


### Вывод

Самая низкое число не вернувших кредит вовремя при высоком доходе (таких людей 7,1%)
А вот самы высокий процент просрочивших платеж при уровне дохода от 149 тыс до 200 тыс (таких 8,8%).

Скорее всего, они привыкли тратить много, хотят и стремятся получать больше, но немножко забывают про свои долги

<a name="step3.4"></a>
- Как разные цели кредита влияют на его возврат в срок?

In [70]:
purpose_debt = clients.pivot_table(values='gender', columns='debt', index='category_purpose', aggfunc='count')

purpose_debt.columns = ['В срок', 'Просрочил']
purpose_debt['Всего'] = purpose_debt['В срок'] + purpose_debt['Просрочил']
purpose_debt['% просроченных'] = purpose_debt['Просрочил'] / purpose_debt['Всего']

purpose_debt

,В срок,Просрочил,Всего,% просроченных
category_purpose,,,,
автомобиль,3886,400,4286,0.093
недвижимость,9418,744,10162,0.073
образование,3626,370,3996,0.093
ремонт,570,35,605,0.058
свадьба,2137,184,2321,0.079


### Вывод

Самое высокое число должников по категории автомобиль и образование (видимо настолько увлечены покупками, что забывают платить) (таких 9,3%)  
По ремонту и недвижимости наблюдаются самое низкое число не вернувших кредит вовремя (таких 5,8 и 7,3% соответственно). Такие люди будут более степенно подходить к своим решениям (ведь недвижимость и ремонт недвижимости - это уже серьезно)

<a name="step4"></a>
### Шаг 4. Общий вывод

В процессе анализа данных заемщиков было проделано следующее:

1) Заполнены пропуски  

   <b>Проблема</b>: В некоторых строках данных были [пропущены значения](#empty) ежемесячного дохода и количество дней трудового стажа.   
   <b>Решение</b>: Такие данные я заполнила [средним значением](#fill_empty) у людей с той же профессией, тем же возрастом, полом и количеством детей (последнее проверялось только для женщин)
   
2) Заменены данные  

<b>Проблема 1</b>: [Количество детей](#children) равнялось -1 и 20  
<b>Решение 1</b>: Произведена [замена](#change_children) на 1 и 2 соответственно

<b>Проблема 2</b>: [Возраст](#age) некоторых людей был равен 0  
<b>Решение 2</b>: [Удалили](#delete_age) такие строки, так как их мало, а на восстановление потратилось бы большое количество времени

<b>Проблема 3</b>: [Трудовой стаж](#days_employed) мог быть отрицательным дробным и очень большим (более 1000 лет)  
<b>Решение 3</b>: [Заменили](#change_days_employed) в таких строках большие значения на большие значения / 24 (потому что кто-то записал в часах, а не в днях), взяли модуль и округлили до целого

<b>Проблема 4</b>: [Образование](#education) могло быть написано разными способами (высшее и ВЫСШЕЕ)  
<b>Решение 4</b>: [Привели](#change_education) все значения поля к нижнему регистру

3) Заменены типы данных
   * [Ежемесячный доход](#change_income) float -> int
   * [Трудовой стаж](#change_days_employed) float -> int
    
4) Удалены дубликаты 
   * [21424 штук -> 21370 штук](#step2.3)
   
5) Лемматизация
   * По полю [цель кредита](#step2.4)

6) Категоризация
   * В поле [цель кредита](#cat_purpose) выделены категории: недвижимость, ремонт, образование, автомобиль, свадьба
   * В поле [доходы](#cat_income) люди разделены по квантилям (от 0 до 25, от 26 до 50, от 51 до 75, от 76 до 100)
   
7) Выявлены закономерности:
   * Между [количеством детей и возвратом кредита в срок](#step3.1)
   * Между [семейным положением и возвратом кредита в срок](#step3.2)
   * Между [уровнем дохода и возвратом кредита в срок](#step3.3)
   * Между [целью кредита и возвратом кредита в срок](#step3.4)